<a href="https://colab.research.google.com/github/mydreamisto/notebooks/blob/main/course/en/chapter3/section3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning a model with the Trainer API or Keras

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets

In [ ]:
!pip install wandb

In [ ]:
!pip install evaluate

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training

In [4]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator
)

In [12]:
import wandb
"""
导入了 wandb 库，这是一个用于实验跟踪和可视化的工具。
wandb.init() 是 wandb 库的初始化函数，它会创建一个新的实验运行，设置实验的一些初始信息，如实验名称、超参数等。
如果不调用这个函数，当后续代码中调用 wandb 的其他函数（如 wandb.log()）时，程序将无法正常工作，因为它不知道将数据记录到哪个实验中。
然后，调用 trainer.train() 进行训练。在训练过程中，程序可能会使用 wandb 来记录训练的各种信息，例如损失、指标等，通过调用 wandb.log() 来实现。
由于我们已经调用了 wandb.init()，所以 wandb.log() 可以正常工作，将数据发送到 wandb 的服务器进行存储和可视化。
"""
wandb.init()

trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.206700
1000,0.123600


TrainOutput(global_step=1377, training_loss=0.1310880476786074, metrics={'train_runtime': 208.0578, 'train_samples_per_second': 52.889, 'train_steps_per_second': 6.618, 'total_flos': 405114969714960.0, 'train_loss': 0.1310880476786074, 'epoch': 3.0})

In [13]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [15]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [16]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8602941176470589, 'f1': 0.9015544041450777}

In [17]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-18-415be425bb02>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.434373,0.803922,0.859649
2,0.572700,0.412914,0.848039,0.891608
3,0.375600,0.676060,0.835784,0.886633


TrainOutput(global_step=1377, training_loss=0.41641840384157136, metrics={'train_runtime': 226.2499, 'train_samples_per_second': 48.636, 'train_steps_per_second': 6.086, 'total_flos': 405114969714960.0, 'train_loss': 0.41641840384157136, 'epoch': 3.0})